In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from __future__ import absolute_import, division, print_function, unicode_literals
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf
os.getcwd()
os.listdir('.')
print(os.getcwd())
print(os.chdir('/content/drive/MyDrive/Proj_colab'))


/content
None


In [3]:
# %% Read the original data and drop the columns
originalD = pd.read_csv('data/Original_data.csv', low_memory=False)
#originalD = pd.read_csv('data/Original_data.csv', low_memory=False)
original_F = originalD.drop(['birthyr','faminc','employ','marstat','child18','pid3','pid7','votereg','presvote16post','ideo5','newsint','inputstate'], axis = 1)
original_F

,id,gender,race,educ,region
0,371823339,1,1,2,2
1,398212310,1,1,2,3
2,392933925,1,1,1,1
3,372445135,1,1,2,2
4,392602384,1,1,2,3
...,...,...,...,...,...
4995,287972460,2,6,2,2
4996,137306469,2,6,2,3
4997,133663305,2,5,5,2
4998,77524502,2,7,6,3


In [4]:
# %% Read the breached  data and drop the columns
breachD = pd.read_csv('data/breached_data.csv', low_memory=False)
breach_F = breachD.drop(['Title','Domain','Name','BreachDate','AddedDate','ModifiedDate','IsSpamList','IsVerified','IsFabricated','IsSensitive','IsActive','IsRetired','PwnCount','LogoType','DataClasses'], axis = 1)
breach_F.loc[:,'Breached'] ='1'
breach_F["Breached"] = breach_F["Breached"].astype(object).astype(int)
breach_F

,id,Breached
0,135664815,1
1,355286483,1
2,355286483,1
3,355286483,1
4,339141795,1
...,...,...
14974,131884325,1
14975,131884325,1
14976,131884325,1
14977,131884325,1


In [5]:
breach_F1 = breach_F.drop_duplicates(subset =["id"] )
breach_F1["Breached"].replace({1: 0},inplace = True)
#df["column1"].replace({"a": "x", "b": "y"}, inplace=True)
#breach_F = breach_
#breach_F.loc[:,'Breached'] ='1'
breach_F1

,id,Breached
0,135664815,0
1,355286483,0
4,339141795,0
5,341961164,0
6,374206867,0
...,...,...
14960,137327203,0
14963,334328189,0
14967,151192859,0
14973,152094711,0


In [6]:
df3 = pd.merge(breach_F, breach_F1, how='outer')
df3

,id,Breached
0,135664815,1
1,355286483,1
2,355286483,1
3,355286483,1
4,339141795,1
...,...,...
19116,137327203,0
19117,334328189,0
19118,151192859,0
19119,152094711,0


In [7]:
# %% Merge the two files
fin_dat = pd.merge(original_F, df3, on='id', how='inner')
print("Number of rows in the final dataset: ", fin_dat.shape[0])
fin_dat.head(5)

Number of rows in the final dataset:  19121


,id,gender,race,educ,region,Breached
0,371823339,1,1,2,2,1
1,371823339,1,1,2,2,1
2,371823339,1,1,2,2,0
3,392933925,1,1,1,1,1
4,392933925,1,1,1,1,1


In [8]:
#input
x=fin_dat.drop('Breached',axis=1)
y= fin_dat.Breached
#splitting
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
#printing shapes of testing and training sets :
print("shape of original dataset :", fin_dat.shape)
print("shape of input - training set", x_train.shape)
print("shape of output - training set", y_train.shape)
print("shape of input - testing set", x_test.shape)
print("shape of output - testing set", y_test.shape)

shape of original dataset : (19121, 6)
shape of input - training set (15296, 5)
shape of output - training set (15296,)
shape of input - testing set (3825, 5)
shape of output - testing set (3825,)


In [27]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [28]:
CATEGORICAL_COLUMNS = ['gender', 'race', 'educ', 'region']
#NUMERIC_COLUMNS = ['age', 'fare']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = x_train[feature_name].unique()  # gets a list of all unique values from given feature column
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

#for feature_name in NUMERIC_COLUMNS:
 # feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

print(feature_columns)

[VocabularyListCategoricalColumn(key='gender', vocabulary_list=(1, 2), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='race', vocabulary_list=(1, 3, 6, 2, 7, 4, 5, 8), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='educ', vocabulary_list=(3, 5, 2, 1, 4, 6), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='region', vocabulary_list=(4, 2, 1, 3), dtype=tf.int64, default_value=-1, num_oov_buckets=0)]


In [29]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in x_train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='id', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='gender', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='race', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='educ', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='region', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [30]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpvhfhaw17', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [31]:
classifier.train(
    input_fn=lambda: input_fn(x_train, y_train, training=True),
    steps=5000)
# We include a lambda to avoid creating an inner function previously

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpvhfhaw17/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 9896143.0, step = 0
INFO:tensorflow:global_step/sec: 389.567
INFO:tensorflow:loss = 4064649.5, step = 100 (0.259 sec)
INFO:tensorflow:global_step/sec: 476.3
INFO:tensorflow:loss = 2230275.0, step = 200 (0.209 sec)
INFO:tensorflow:global_step/sec: 518.609
INFO:tensorflow:loss = 1004735.06, step = 300 (0.195 sec)
INFO:tensorflow:global_step/sec: 502.219
INFO:tensorflow:loss = 112867.3, step = 400 (0.197 sec)
INFO:tensorflow:global_step/sec: 519.792
INFO:tensorflow:loss = 39666.848, step = 500 (0.195 sec)
INFO:ten

In [32]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(x_test, y_test, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-12-01T04:56:30Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpvhfhaw17/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.23485s
INFO:tensorflow:Finished evaluation at 2020-12-01-04:56:30
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.78300655, average_loss = 124.95289, global_step = 5000, loss = 124.96261
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmpvhfhaw17/model.ckpt-5000

Test set accuracy: 0.783



In [ ]:
def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['gender','race','educ', 'region']
predict = {}

print("Please type numeric values as prompted.")
for feature in features:
  valid = True
  while valid: 
    val = input(feature + ": ")
    if not val.isdigit(): valid = False

  predict[feature] = [int(val)]

predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(
        y_train[class_id], 100 * probability))

In [ ]:
# Here is some example input and expected classes you can try above
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}